In [132]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [48]:
# saving the jhon hopkins dala link
JH_url_in =  "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"

In [49]:
# saving the jhon hopkins file names
JH_file_names = [
    "time_series_covid19_confirmed_global.csv",
    "time_series_covid19_deaths_global.csv"
]

In [50]:
# joining the link to names
JH_URL = [JH_url_in + filename for filename in JH_file_names]

In [51]:
# saving link for population data
JH_UID = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv"

In [52]:
# saving the link to containment data
OX_url_in = "https://raw.githubusercontent.com/OxCGRT/covid-policy-dataset/main/data/timeseries_indices/OxCGRT_timeseries_ContainmentHealthIndex_v1.csv"

In [53]:
# loading data from links
global_cases = pd.read_csv(JH_URL[0]) #reading global cases
global_deaths = pd.read_csv(JH_URL[1]) # reding global death
UID = pd.read_csv(JH_UID) # reading world population by country
CONT = pd.read_csv (OX_url_in) # reading Containment and closure Policies by country

In [54]:
# Pivot the global cases DataFrame to long format
global_cases = global_cases.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                                 var_name='date', 
                                 value_name='cases')

# Select columns, excluding 'Province/State'
global_cases = global_cases[['Country/Region', 'date', 'cases', 'Lat', 'Long']]

In [55]:
# Convert the 'date' column to datetime format
global_cases['date'] = pd.to_datetime(global_cases['date'])

In [56]:
# Sort the DataFrame by 'Country/Region'
global_cases = global_cases.sort_values(by=['Country/Region', 'date'])

# Reset the index (optional)
global_cases = global_cases.reset_index(drop=True)

In [57]:
# Pivot the global deaths DataFrame to long format
id_vars = ['Country/Region', 'Province/State', 'Lat', 'Long']

# Pivot the DataFrame using pd.melt()
global_deaths = pd.melt(global_deaths, id_vars=id_vars, var_name='date', value_name='deaths')

# Select columns to keep
global_deaths = global_deaths[['Country/Region', 'date', 'deaths']]

In [58]:
# Convert the 'date' column to datetime format
global_deaths['date'] = pd.to_datetime(global_deaths['date'])

In [59]:
# Sort the DataFrame by 'Country/Region'
global_deaths = global_deaths.sort_values(by=['Country/Region', 'date'])

# Reset the index (optional)
global_deaths = global_deaths.reset_index(drop=True)

In [60]:
# Tidy population data

In [61]:
# Select columns 'Province_State', 'Country_Region', 'Population'
UID = UID[['Province_State', 'Country_Region', 'Population']]

# Drop rows with missing values in the 'Population' column
UID = UID.dropna(subset=['Population'])

# Group by 'Country_Region' and calculate the maximum population
UID = UID.groupby('Country_Region').agg({'Population': 'max'}).reset_index()

# Rename the columns if needed
UID.columns = ['Country_Region', 'Population']

In [62]:
# Tidy the containment and health indices data

# Columns to keep as identifiers
id_vars = ['CountryCode', 'CountryName', 'RegionCode', 'RegionName', 'CityCode', 'CityName', 'Jurisdiction']

# Pivot the DataFrame using pd.melt()
CONT = pd.melt(CONT, id_vars=id_vars, var_name='date', value_name='Containment')

# Select columns to keep
CONT = CONT[['CountryName', 'date', 'Containment']]


In [63]:
# Convert the 'date' column to datetime format
CONT['date'] = pd.to_datetime(CONT['date'])

In [64]:
# Sort the DataFrame by 'Country/Region and date'
CONT = CONT.sort_values(by=['CountryName', 'date'])

# Reset the index (optional)
CONT = CONT.reset_index(drop=True)

In [65]:
# unify the names

CONT = CONT.rename(columns={'CountryName': 'Country'})
global_cases = global_cases.rename(columns={'Country/Region': 'Country'})
global_deaths = global_deaths.rename(columns={'Country/Region': 'Country'})
UID = UID.rename(columns={'Country_Region': 'Country'})

In [66]:
# merging global cases with global death
Global = pd.merge(global_cases, global_deaths, on=['Country', 'date'], how='inner')

In [67]:
Global

,Country,date,cases,Lat,Long,deaths
0,Afghanistan,2020-01-22,0,33.939110,67.709953,0
1,Afghanistan,2020-01-23,0,33.939110,67.709953,0
2,Afghanistan,2020-01-24,0,33.939110,67.709953,0
3,Afghanistan,2020-01-25,0,33.939110,67.709953,0
4,Afghanistan,2020-01-26,0,33.939110,67.709953,0
...,...,...,...,...,...,...
2378578,Zimbabwe,2023-03-05,264127,-19.015438,29.154857,5668
2378579,Zimbabwe,2023-03-06,264127,-19.015438,29.154857,5668
2378580,Zimbabwe,2023-03-07,264127,-19.015438,29.154857,5668
2378581,Zimbabwe,2023-03-08,264276,-19.015438,29.154857,5671


In [68]:
# merging global with CONT

Global = pd.merge(Global, CONT, how='left', on=['Country', 'date'])


In [69]:
Global

,Country,date,cases,Lat,Long,deaths,Containment
0,Afghanistan,2020-01-22,0,33.939110,67.709953,0,0.0
1,Afghanistan,2020-01-23,0,33.939110,67.709953,0,0.0
2,Afghanistan,2020-01-24,0,33.939110,67.709953,0,0.0
3,Afghanistan,2020-01-25,0,33.939110,67.709953,0,0.0
4,Afghanistan,2020-01-26,0,33.939110,67.709953,0,0.0
...,...,...,...,...,...,...,...
49508752,Zimbabwe,2023-03-05,264127,-19.015438,29.154857,5668,NaN
49508753,Zimbabwe,2023-03-06,264127,-19.015438,29.154857,5668,NaN
49508754,Zimbabwe,2023-03-07,264127,-19.015438,29.154857,5668,NaN
49508755,Zimbabwe,2023-03-08,264276,-19.015438,29.154857,5671,NaN


In [70]:
# merging global with Polulation

Global = pd.merge(Global, UID, how='left', on=['Country'])

In [71]:
Global

,Country,date,cases,Lat,Long,deaths,Containment,Population
0,Afghanistan,2020-01-22,0,33.939110,67.709953,0,0.0,38928341.0
1,Afghanistan,2020-01-23,0,33.939110,67.709953,0,0.0,38928341.0
2,Afghanistan,2020-01-24,0,33.939110,67.709953,0,0.0,38928341.0
3,Afghanistan,2020-01-25,0,33.939110,67.709953,0,0.0,38928341.0
4,Afghanistan,2020-01-26,0,33.939110,67.709953,0,0.0,38928341.0
...,...,...,...,...,...,...,...,...
49508752,Zimbabwe,2023-03-05,264127,-19.015438,29.154857,5668,NaN,14862927.0
49508753,Zimbabwe,2023-03-06,264127,-19.015438,29.154857,5668,NaN,14862927.0
49508754,Zimbabwe,2023-03-07,264127,-19.015438,29.154857,5668,NaN,14862927.0
49508755,Zimbabwe,2023-03-08,264276,-19.015438,29.154857,5671,NaN,14862927.0


In [72]:
# Group by 'Country_Region' and 'date', and calculate the maximum values
global_sum = Global.groupby(['Country']).agg({
    'deaths': 'max',
    'cases': 'max',
    'Population': 'max',
    'Containment': 'mean',
    'Lat':'max',
    'Long':'max'
}).reset_index()

In [73]:
global_sum

,Country,deaths,cases,Population,Containment,Lat,Long
0,Afghanistan,7896,209451,38928341.0,30.294335,33.939110,67.709953
1,Albania,3598,334457,2877800.0,47.095619,41.153300,20.168300
2,Algeria,6881,271496,43851043.0,50.632912,28.033900,1.659600
3,Andorra,165,47890,77265.0,46.271070,42.506300,1.521800
4,Angola,1933,105288,32866268.0,50.021879,-11.202700,17.873900
...,...,...,...,...,...,...,...
196,West Bank and Gaza,5708,703228,5101416.0,NaN,31.952200,35.233200
197,Winter Olympics 2022,0,535,NaN,NaN,39.904200,116.407400
198,Yemen,2159,11945,29825968.0,24.422456,15.552727,48.516388
199,Zambia,4057,343135,18383956.0,36.726493,-13.133897,27.849332


In [74]:
# Calculate 'Death per million' and assign it to a new column
global_sum['death_per_million'] = (global_sum['deaths'] / global_sum['Population']) * 1000000

In [75]:
# Calculate 'Cases per million' and assign it to a new column
global_sum['cases_per_million'] = (global_sum['cases'] / global_sum['Population']) * 1000000

In [76]:
# Calculate 'percentage death per cases' and assign it to a new column
global_sum['percent_death_per_cases'] = (global_sum['deaths'] / global_sum['cases']) * 100

In [77]:
# Deleting rows with no data
global_sum = global_sum.dropna(how='any')

# Round the 'death_per_million' column to 2 decimal places
global_sum['death_per_million'] = global_sum['death_per_million'].round().astype(int)
global_sum['cases_per_million'] = global_sum['cases_per_million'].round().astype(int)
global_sum['Containment'] = global_sum['Containment'].round().astype(int)
global_sum['Population'] = global_sum['Population'].round().astype(int)
global_sum['percent_death_per_cases'] = global_sum['percent_death_per_cases'].round(1)



/var/folders/65/520qd96s3vbfvck0c8npy5j40000gn/T/ipykernel_11516/1901360828.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/65/520qd96s3vbfvck0c8npy5j40000gn/T/ipykernel_11516/1901360828.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/65/520qd96s3vbfvck0c8npy5j40000gn/T/ipykernel_11516/1901360828.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [78]:
global_sum

,Country,deaths,cases,Population,Containment,Lat,Long,death_per_million,cases_per_million,percent_death_per_cases
0,Afghanistan,7896,209451,38928341,30,33.939110,67.709953,203,5380,3.8
1,Albania,3598,334457,2877800,47,41.153300,20.168300,1250,116220,1.1
2,Algeria,6881,271496,43851043,51,28.033900,1.659600,157,6191,2.5
3,Andorra,165,47890,77265,46,42.506300,1.521800,2136,619815,0.3
4,Angola,1933,105288,32866268,50,-11.202700,17.873900,59,3204,1.8
...,...,...,...,...,...,...,...,...,...,...
194,Venezuela,5854,552162,28435943,53,6.423800,-66.589700,206,19418,1.1
195,Vietnam,43186,11526994,97338583,59,14.058324,108.277199,444,118422,0.4
198,Yemen,2159,11945,29825968,24,15.552727,48.516388,72,400,18.1
199,Zambia,4057,343135,18383956,37,-13.133897,27.849332,221,18665,1.2


In [124]:
df = pd.DataFrame(global_sum)

In [134]:
fig = px.scatter_geo(df,
                     lat='Lat',
                     lon='Long',
                     size='percent_death_per_cases',
                     projection='natural earth',
                     title='World Map with Covid_19 Data ',
                     hover_name='Country',  # Hover information
                     color='Containment',  # Color based on 'Value' column
                     color_continuous_scale='Inferno'  # Color scale
                     )
fig.update_traces(hovertemplate='Country: %{hovertext}<br>Death Rate: %{marker.size}<br>Containment: %{marker.color}')


legend_entry = go.layout.Annotation(
    text='Circle Size Represents<br>Death Rate (%)',
    showarrow=False,
    xref='paper',
    yref='paper',
    x=0.95,
    y=1.1,
    xanchor='left',
    yanchor='top',
    bgcolor='white',
    bordercolor='black',
    borderwidth=1,
    font=dict(size=12),
)

# Add the custom legend entry to the layout
fig.update_layout(annotations=[legend_entry])

In [133]:
# Assuming 'fig' is your Plotly figure
# Specify the file name and path where you want to save the HTML file
html_file_path = 'your_map.html'

# Use the write_html function to export the figure as an HTML file
pio.write_html(fig, file=html_file_path, auto_open=True)


In [1]:
pip install voila

     |████████████████████████████████| 3.3 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 2.2 MB/s eta 0:00:011
     |████████████████████████████████| 104 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 375 kB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 254 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 423 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 333 kB 7.7 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 5.9 MB/s eta 0:00:011
     |████████████████████████████████| 57 kB 6.8 MB/s eta 0:00:011
     |████████████████████████████████| 10.1 MB 2.5 MB/s eta 0:00:01    |██████████████▏                 | 4.5 MB 3.3

     |████████████████████████████████| 77 kB 8.0 MB/s  eta 0:00:01
  Attempting uninstall: attrs
    Found existing installation: attrs 21.4.0
    Uninstalling attrs-21.4.0:
      Successfully uninstalled attrs-21.4.0
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.1.1
    Uninstalling traitlets-5.1.1:
      Successfully uninstalled traitlets-5.1.1
  Attempting uninstall: tornado
    Found existing installation: tornado 6.1
    Uninstalling tornado-6.1:
      Successfully uninstalled tornado-6.1
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 22.3.0
    Uninstalling pyzmq-22.3.0:
      Successfully uninstalled pyzmq-22.3.0
  Attempting uninstall: jupyter-core
    Found existing installation: jupyter-core 4.9.2
    Uninstalling jupyter-core-4.9.2:
      Successfully uninstalled jupyter-core-4.9.2
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.4.0
    Uninstalling jsonschema-4.4.0:
      Successfully u

In [2]:
pip install voila-material 


     |████████████████████████████████| 161 kB 671 kB/s eta 0:00:01


Note: you may need to restart the kernel to use updated packages.


In [1]:
voila nameofnotebook.ipynb 

SyntaxError: invalid syntax (3632626157.py, line 1)